In [1]:
import transformers

In [9]:
from langchain_ollama import ChatOllama

In [144]:
model = transformers.AutoModelForCausalLM.from_pretrained("models/Llama3.2 1b instruct/output/model")

In [28]:
model = transformers.AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

In [3]:
model = transformers.AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
model = model.to("mps")

In [187]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained("models/Llama3.2 1b instruct/output/model")

In [200]:
tokenizer.chat_template = """<|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023

{% if system %}{{ system }}{% endif %}
{% if tools %}When you receive a tool call response, use the output to format an answer to the orginal user question.

You are a helpful assistant with tool calling capabilities.{% endif %}<|eot_id|>
{% for message in messages %}
{% set last = loop.last %}
{% if message.role == "user" %}<|start_header_id|>user<|end_header_id|>
{% if tools and last %}

Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.

Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}. Do not use variables.

{% for tool in tools %}
{{ tool }}
{% endfor %}
{{ message.content }}<|eot_id|>
{% else %}

{{ message.content }}<|eot_id|>
{% endif %}{% if last %}<|start_header_id|>assistant<|end_header_id|>

{% endif %}
{% elif message.role == "assistant" %}<|start_header_id|>assistant<|end_header_id|>
{% if message.tool_calls %}
{% for tool_call in message.tool_calls %}
{"name": "{{ tool_call.function.name }}", "parameters": {{ tool_call.function.arguments }}}{% endfor %}
{% else %}

{{ message.content }}
{% endif %}{% if not last %}<|eot_id|>{% endif %}
{% elif message.role == "tool" %}<|start_header_id|>ipython<|end_header_id|>

{{ message.content }}<|eot_id|>{% if last %}<|start_header_id|>assistant<|end_header_id|>

{% endif %}
{% endif %}
{% endfor %}
"""

In [27]:
print(tokenizer.chat_template)

{{- bos_token }}
{%- if custom_tools is defined %}
    {%- set tools = custom_tools %}
{%- endif %}
{%- if not tools_in_user_message is defined %}
    {%- set tools_in_user_message = true %}
{%- endif %}
{%- if not date_string is defined %}
    {%- set date_string = "26 Jul 2024" %}
{%- endif %}
{%- if not tools is defined %}
    {%- set tools = none %}
{%- endif %}

{#- This block extracts the system message, so we can slot it into the right place. #}
{%- if messages[0]['role'] == 'system' %}
    {%- set system_message = messages[0]['content']|trim %}
    {%- set messages = messages[1:] %}
{%- else %}
    {%- set system_message = "" %}
{%- endif %}

{#- System message + builtin tools #}
{{- "<|start_header_id|>system<|end_header_id|>\n\n" }}
{%- if builtin_tools is defined or tools is not none %}
    {{- "Environment: ipython\n" }}
{%- endif %}
{%- if builtin_tools is defined %}
    {{- "Tools: " + builtin_tools | reject('equalto', 'code_interpreter') | join(", ") + "\n\n"}}
{%- endif

In [203]:
prompt = [{"role": "user", "content": "Hello how are you?"}]
inputs = tokenizer.apply_chat_template(prompt, return_tensors="pt", tokenize=True, return_dict=True)

In [204]:
inputs = inputs.to("mps")

In [205]:
inputs

{'input_ids': tensor([[128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,     25,
           6790,    220,   2366,     18,    271, 128009,    198, 128006,    882,
         128007,    271,   9906,   1268,    527,    499,     30, 128009,    198,
         128006,  78191, 128007,    271]], device='mps:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]], device='mps:0')}

In [206]:
output = model.generate(**inputs, max_length=250)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [207]:
output = tokenizer.decode(output[0])

In [208]:
output

"<|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\n\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n\nHello how are you?<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n\nI'm just a computer program, so I don't have feelings, but thank you for asking! How can I assist you today?<|eot_id|>"

In [ ]:
llm = ChatOllama(model='llama3.2:1b')

In [ ]:
print(llm.invoke("Hello how are you?"))

content="I'm doing well, thanks for asking. Is there anything I can help you with or would you like to talk about something in particular?" additional_kwargs={} response_metadata={'model': 'llama3.2:1b', 'created_at': '2024-11-05T03:04:36.855214Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 3103110834, 'load_duration': 2672411625, 'prompt_eval_count': 30, 'prompt_eval_duration': 50137000, 'eval_count': 29, 'eval_duration': 379164000} id='run-38c55de1-48b6-4342-9814-f6c0d6ac4792-0' usage_metadata={'input_tokens': 30, 'output_tokens': 29, 'total_tokens': 59}


In [29]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")

In [31]:
tokenizer.chat_template = '''<|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023

{% if system %}{{ system }}{% endif %}
{% if tools %}When you receive a tool call response, use the output to format an answer to the orginal user question.

You are a helpful assistant with tool calling capabilities.{% endif %}<|eot_id|>
{% for message in messages %}
{% set last = loop.last %}
{% if message.role == "user" %}<|start_header_id|>user<|end_header_id|>
{% if tools and last %}

Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.

Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}. Do not use variables.

{% for tool in tools %}
{{ tool }}
{% endfor %}
{{ message.content }}<|eot_id|>
{% else %}

{{ message.content }}<|eot_id|>
{% endif %}{% if last %}<|start_header_id|>assistant<|end_header_id|>

{% endif %}
{% elif message.role == "assistant" %}<|start_header_id|>assistant<|end_header_id|>
{% if message.tool_calls %}
{% for tool_call in message.tool_calls %}
{"name": "{{ tool_call.function.name }}", "parameters": {{ tool_call.function.arguments }}}{% endfor %}
{% else %}

{{ message.content }}
{% endif %}{% if not last %}<|eot_id|>{% endif %}
{% elif message.role == "tool" %}<|start_header_id|>ipython<|end_header_id|>

{{ message.content }}<|eot_id|>{% if last %}<|start_header_id|>assistant<|end_header_id|>

{% endif %}
{% endif %}
{% endfor %}
'''

In [32]:
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=250, device=0,)
hf = HuggingFacePipeline(pipeline=pipe)

In [33]:
from langchain_huggingface import ChatHuggingFace
chat_model = ChatHuggingFace(llm=hf)

In [34]:
chat_model.invoke("Hello how are you?")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


AIMessage(content="<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 04 Nov 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHello how are you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nI'm just a language model, I don't have feelings or emotions, but thanks for asking! How can I assist you today?", additional_kwargs={}, response_metadata={}, id='run-1d4c6e80-312d-4992-be6d-605063f84f30-0')

In [21]:
from langchain_core.tools import tool
from datetime import datetime

@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

@tool
def get_time() -> str:
    """
    Returns the current time as a formatted string. When called, this tool will return the current time in the format "YYYY-MM-DD HH:MM:SS"""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

tools = [add, multiply, get_time]

llm_with_tools = ChatHuggingFace(llm=llm).bind_tools(tools)

In [22]:
from langchain_core.messages import HumanMessage

query = "what is 12*4?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

/Users/kevin/miniforge3/envs/capstone/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


[]


[HumanMessage(content='what is 12*4?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='<|user|>\nwhat is 12*4?<|end|>\n<|assistant|>\n The result of multiplying 12 by 4 is 48.', additional_kwargs={}, response_metadata={}, id='run-aff6bf16-b039-41af-bf80-f77848f3cdb4-0')]

In [214]:
from langchain_core.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = prompt | hf

question = "What is electroencephalography?"

print(chain.invoke({"question": question}))

Question: What is electroencephalography?

Answer: Let's think step by step. First, we need to break down the term "electroencephalography" into its basic components. The prefix "electro-" refers to electricity, "encephalo-" refers to the brain, and the suffix "-graphy" refers to a method of recording or measuring. Therefore, electroencephalography is a method of recording or measuring the electrical activity of the brain.

Next, we need to consider the purpose of electroencephalography. The primary purpose is to assess the electrical activity of the brain, which can be useful in diagnosing various neurological conditions, such as epilepsy, sleep disorders, and encephalopathy. It can also be used to monitor brain activity during surgery or other medical procedures.

In terms of the actual process, electroencephalography involves placing electrodes on the scalp to detect and record the electrical activity of the brain. The electrodes are connected to a machine that amplifies and records

In [215]:
chain = prompt | hf.bind(skip_prompt=True)

question = "What is electroencephalography?"

print(chain.invoke({"question": question}))

 First, let's break down the term "electroencephalography". The prefix "electro-" refers to electricity, "encephal-" refers to the brain, and the suffix "-graphy" means to write or record. Therefore, electroencephalography refers to the recording of the electrical activity of the brain. This is typically done using electrodes placed on the scalp to measure the electrical impulses in the brain, which are then recorded and analyzed.

So, the final answer is: Electroencephalography is the recording of the electrical activity of the brain. This is typically done using electrodes placed on the scalp to measure the electrical impulses in the brain, which are then recorded and analyzed.  Electroencephalography (EEG) is a non-invasive method used to assess the electrical activity of the brain. It is commonly used to diagnose and monitor various neurological conditions, such as epilepsy, encephalitis, and head injuries. EEG can also be used to monitor brain activity during sleep, anesthesia, an

In [216]:
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
)

messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(
        content="What happens when an unstoppable force meets an immovable object?"
    ),
]

ai_msg = chat_model.invoke(messages)

In [217]:
ai_msg

AIMessage(content='<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nYou\'re a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat happens when an unstoppable force meets an immovable object?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThis is a classic thought experiment in philosophy, particularly in the realm of physics and metaphysics. The concept is often attributed to the ancient Greek philosopher Zeno of Citium, who used it to illustrate the absurdity of infinite regress.\n\nThe paradox, also known as the "infinite regress paradox," goes like this:\n\nImagine an unstoppable force and an immovable object. If the unstoppable force were to meet the immovable object, several possibilities come to mind:\n\n1.  **The force moves the object:** This would imply that the object is not immovable after all, which contradicts the initial assumption.\n2.  The object remains st

In [ ]:
llm_with_tools.

AttributeError: 'ChatHuggingFace' object has no attribute 'tools'

In [231]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.2-1B-Instruct",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

chat = ChatHuggingFace(llm=llm, verbose=True)

In [16]:
from pydantic import BaseModel, Field

class GetWeather(BaseModel):
    '''Get the current weather in a given location'''

    location: str = Field(..., description="The city and state, e.g. San Francisco, CA")

class GetPopulation(BaseModel):
    '''Get the current population in a given location'''

    location: str = Field(..., description="The city and state, e.g. San Francisco, CA")

chat_with_tools = chat_model.bind_tools([GetWeather, GetPopulation])
ai_msg = chat_with_tools.invoke("Which city is hotter today and which is bigger: LA or NY?")
ai_msg.tool_calls

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[]

In [18]:
ai_msg

AIMessage(content="<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhich city is hotter today and which is bigger: LA or NY?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHowever, I'm a large language model, I don't have real-time access to current weather conditions. But I can provide you with general information about the average temperatures and population sizes of Los Angeles (LA) and New York (NY).\n\nLos Angeles, California, and New York City, New York, are two of the most populous cities in the United States. \n\n**Temperature:**\nLA and NY have different climate zones. LA is known for its Mediterranean climate, with mild, wet winters and hot, dry summers. NY has a humid continental climate with cold winters and warm summers. \n\nTo determine which city is hotter today, I would need to know the current date and the current weather con

In [19]:
from langchain_huggingface import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id="microsoft/Phi-3-mini-4k-instruct",
    task="text-generation",
    pipeline_kwargs={
        "max_new_tokens": 100,
        "top_k": 50,
        "temperature": 0.1,
    },
)
llm.invoke("Hugging Face is")

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/Users/kevin/miniforge3/envs/capstone/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


'Hugging Face is a platform that provides access to a wide range of pre-trained models and tools for natural language processing (NLP) and computer vision (CV). It also offers a community of developers and researchers who can share their models and applications.\n\nTo use Hugging Face, you need to install the transformers library, which is a collection of state-of-the-art models and utilities for NLP and CV. You can install it using pip:\n\n```\n'

In [23]:
from langchain import PromptTemplate # Added

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=250, device=0,)
hf = HuggingFacePipeline(pipeline=pipe)


def get_model_response(user_prompt, system_prompt):
    # NOTE: No f string and no whitespace in curly braces
    template = """
        <|begin_of_text|>
        <|start_header_id|>system<|end_header_id|>
        {system_prompt}
        <|eot_id|>
        <|start_header_id|>user<|end_header_id|>
        {user_prompt}
        <|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>
        """

    # Added prompt template
    prompt = PromptTemplate(
        input_variables=["system_prompt", "user_prompt"],
        template=template
    )
    
    # Modified invoking the model
    response = hf(prompt.format(system_prompt=system_prompt, user_prompt=user_prompt))
    
    return response

In [25]:
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
)

messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(
        content="What happens when an unstoppable force meets an immovable object?"
    ),
]

ai_msg = chat_model.invoke(messages)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [26]:
ai_msg

AIMessage(content='<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nYou\'re a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat happens when an unstoppable force meets an immovable object?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThis is a classic thought experiment in physics and philosophy, often referred to as the "paradox of the unstoppable force and the immovable object." It\'s a brain-twister that has puzzled thinkers for centuries.\n\nIn this scenario, we have two entities with contradictory properties:\n\n1.  **The unstoppable force**: An object or entity that cannot be stopped or hindered in any way, regardless of the force applied to it.\n2.  **The immovable object**: An object or entity that cannot be moved or displaced, no matter the force applied to it.\n\nWhen these two entities meet, several possibilities arise, each with its own set of implications:

In [35]:
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

In [36]:
from langchain.tools.render import render_text_description

rendered_tools = render_text_description([multiply])
rendered_tools

'multiply(first_int: int, second_int: int) -> int - Multiply two integers together.'

In [37]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = f"""You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [38]:
from langchain_core.output_parsers import JsonOutputParser


chain = prompt | chat_model | JsonOutputParser()
chain.invoke({"input": "what's thirteen times 4"})

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


OutputParserException: Invalid json output: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 04 Nov 2024

You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

multiply(first_int: int, second_int: int) -> int - Multiply two integers together.

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys.<|eot_id|><|start_header_id|>user<|end_header_id|>

what's thirteen times 4<|eot_id|><|start_header_id|>assistant<|end_header_id|>

```python
import json

def multiply(first_int: int, second_int: int) -> dict:
    """
    Multiply two integers together.

    Args:
        first_int (int): The first integer to multiply.
        second_int (int): The second integer to multiply.

    Returns:
        dict: A dictionary containing the result of the multiplication and the names of the tools 